In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110600  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,7.933222e+06
2018-02-28,6.674263e+06
2018-03-31,8.163427e+06
2018-04-30,7.752493e+06
2018-05-31,7.884976e+06
2018-06-30,7.498588e+06
2018-07-31,8.179373e+06
2018-08-31,7.949378e+06
2018-09-30,7.935168e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    7.933222e+06
 2018-02-28    6.674263e+06
 2018-03-31    8.163427e+06
 2018-04-30    7.752493e+06
 2018-05-31    7.884976e+06
 2018-06-30    7.498588e+06
 2018-07-31    8.179373e+06
 2018-08-31    7.949378e+06
 2018-09-30    7.935168e+06
 2018-10-31    8.167213e+06
 2018-11-30    7.835971e+06
 2018-12-31    7.868522e+06
 2019-01-31    7.660775e+06
 2019-02-28    6.880081e+06
 2019-03-31    8.276329e+06
 2019-04-30    8.034961e+06
 2019-05-31    8.033693e+06
 2019-06-30    7.753976e+06
 2019-07-31    8.211883e+06
 2019-08-31    7.860153e+06
 2019-09-30    7.878358e+06
 2019-10-31    7.927514e+06
 2019-11-30    8.209251e+06
 2019-12-31    9.949904e+06
 2020-01-31    9.748664e+06
 2020-02-29    8.774334e+06
 2020-03-31    7.180110e+06
 2020-04-30    6.683972e+06
 2020-05-31    7.078121e+06
 2020-06-30    7.619819e+06
 2020-07-31    8.464509e+06
 2020-08-31    7.720841e+06
 2020-09-30    7.512405e+06
 2020-10-31    8.090000e+06
 2020-11-30    7.975433e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -4.301079
p-value : 0.000442
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1245.131, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1238.863, Time=0.11 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1238.824, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1240.194, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1237.731, Time=0.14 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1239.643, Time=0.23 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1238.875, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1240.827, Time=0.07 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1240.995, Time=0.09 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1241.440, Time=0.17 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1239.534, Time=0.41 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1238.788, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1241.744, Time=0.26 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1240.863, Time=0.17 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1241.391, Time=0.30 sec
 ARIMA(3,0

ARIMA(order=(0, 0, 4), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,0,4))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 4)   Log Likelihood                -612.792
Method:                       css-mle   S.D. of innovations         511956.705
Date:                Sat, 11 Sep 2021   AIC                           1237.585
Time:                        02:03:59   BIC                           1248.011
Sample:                             0   HQIC                          1241.406
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       7.878e+06   8.92e+04     88.370      0.000     7.7e+06    8.05e+06
ma.L1.y        0.4492      0.148      3.026      0.002       0.158       0.740
ma.L2.y        0.3110      0.179      1.733      0.0

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([7776112.87137166, 7878488.86130867]), array([511956.70547502, 561239.18274912]), array([[6772696.16699684, 8779529.57574648],
       [6778480.27640771, 8978497.44620963]]))
